# Auto-ML 01 with Auto-Sklearn

In [1]:
%reset -f
from pprint import pprint
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import rcParams
import seaborn as sns
sns.set()
rcParams['figure.figsize'] = 10,10
from sklearn.preprocessing import StandardScaler 
from functionsAll import lstmMultiSplit
import sklearn
import autosklearn.regression as regression

functionsAll.py loaded successfully


In [17]:
df = pd.read_csv("../Data/cleanned/zusammen.csv", parse_dates=["date"])
df['dayofyear'] = df['date'].dt.dayofyear
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df['weekday'] = df['date'].dt.weekday
df['hour'] = df['date'].dt.hour
df['day'] = df['date'].dt.day
df['week'] = df['date'].dt.week
df.drop(columns=['date','rohwert'], inplace= True)
# Conclude with 11 features, 1 label.
df = df[[ 'wert', 'day', 'month', 'dayofyear', 'year', 'weekday', 'week', 'hour','temp', 'dwpt', 'rhum', 'prcp']]
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)

/home/cluster_home/yash/workspace/AutoMLRepo/Auto-ML-On-De-Pollution/autoMLVenv/lib64/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [3]:
df.head(5)

,wert,day,month,dayofyear,year,weekday,week,hour,temp,dwpt,rhum,prcp
0,36,2,1,2,2020,3,1,0,1.0,-0.6,89.0,0.0
1,31,2,1,2,2020,3,1,7,0.1,-1.2,91.0,0.0
2,32,2,1,2,2020,3,1,6,-0.3,-1.3,93.0,0.0
3,27,2,1,2,2020,3,1,5,-0.5,-1.2,95.0,0.0
4,38,2,1,2,2020,3,1,4,-0.4,-0.8,97.0,0.0


In [18]:
y = df.pop('wert')
x = df 
trainX = x.query('index < 23000')
testX = x.query('index >= 23000')
trainY = y[:23000]
testY = y[23000:]
print(trainX.shape, testX.shape, trainY.shape, testY.shape)

(23000, 11) (736, 11) (23000,) (736,)


In [5]:
autoML = regression.AutoSklearnRegressor(time_left_for_this_task=600,per_run_time_limit=60)
autoML.fit(trainX, trainY)

AutoSklearnRegressor(per_run_time_limit=60, time_left_for_this_task=600)

In [10]:
import pickle
with open('../models/aml/automl01.pkl','wb') as f:
    pickle.dump(autoML,f)

In [11]:
predict = autoML.predict(testX)

## Evaluate the model

In [19]:
from sklearn.metrics import mean_absolute_error as MAE, mean_squared_error as RMSE, mean_absolute_percentage_error as MAPE, r2_score as R2
yPred = predict
yTrue = testY

In [20]:
yPred.shape, yTrue.shape

((736,), (736,))

In [22]:
print(f"The MAE (Mean Absolute Error) of model Auto-SK-Learn Auto-ML is : {MAE(yTrue, yPred)}") 
print(f"The RMSE (Root Mean Square Error) of model Auto-SK-Learn Auto-ML is : {RMSE(yTrue, yPred)}")
print(f"The MAPE (Mean Absolute Percentage Error) of model Auto-SK-Learn Auto-ML is : {MAPE(yTrue, yPred)}")
print(f"The R2 (R-Square) of model Auto-SK-Learn Auto-ML is : {R2(yTrue, yPred)}")

The MAE (Mean Absolute Error) of model Auto-SK-Learn Auto-ML is : 4.982609403360149
The RMSE (Root Mean Square Error) of model Auto-SK-Learn Auto-ML is : 43.42150107843412
The MAPE (Mean Absolute Percentage Error) of model Auto-SK-Learn Auto-ML is : 0.31596617632039986
The R2 (R-Square) of model Auto-SK-Learn Auto-ML is : -0.024439125434468245


In [48]:
from pprint import pprint
pprint(autoML.show_models())

('[(1.000000, '
 "SimpleRegressionPipeline({'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': "
 "'no_encoding', "
 "'data_preprocessing:categorical_transformer:category_coalescence:__choice__': "
 "'minority_coalescer', "
 "'data_preprocessing:numerical_transformer:imputation:strategy': 'mean', "
 "'data_preprocessing:numerical_transformer:rescaling:__choice__': "
 "'robust_scaler', 'feature_preprocessor:__choice__': 'no_preprocessing', "
 "'regressor:__choice__': 'k_nearest_neighbors', "
 "'data_preprocessing:categorical_transformer:category_coalescence:minority_coalescer:minimum_fraction': "
 '0.00397868572473159, '
 "'data_preprocessing:numerical_transformer:rescaling:robust_scaler:q_max': "
 '0.7840854273341944, '
 "'data_preprocessing:numerical_transformer:rescaling:robust_scaler:q_min': "
 "0.11376189722146844, 'regressor:k_nearest_neighbors:n_neighbors': 20, "
 "'regressor:k_nearest_neighbors:p': 2, "
 "'regressor:k_nearest_neighbors:weights': 'dista